In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle
import pandas as pd
import numpy as np

data = pickle.load(open("/content/drive/My Drive/New Data/vocab_train.pkl", 'rb'))

In [0]:
!pip install bayesian-optimization

In [0]:
 y= data.iloc[:,200]
X=data.iloc[:,:200]

In [0]:
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
train_data, X_test, train_labels, y_test = train_test_split(X, y, test_size=0.3, random_state=1994, stratify=y)

In [0]:
train_data=train_data.append(train_data[train_labels==1])
train_labels=train_labels.append(train_labels[train_labels==1])

In [0]:
params = {'max_depth': 4,
              'gamma': 0.5,
              'n_estimators': 112,
              'learning_rate':0.05,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              #'min_child_weight' : int(min_child_weight),
              'eta': 0.05,
              'eval_metric': 'auc'}

In [0]:
#Initializing an XGBClassifier with default parameters and fitting the training data
from xgboost import XGBClassifier
classifier1 = XGBClassifier(**params).fit(train_data, train_labels)

In [0]:
#Predicting for training set
train_p1 = classifier1.predict(X_test)


In [0]:

#Printing the classification report
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_p1, y_test))

              precision    recall  f1-score   support

           0       0.99      0.86      0.92    182189
           1       0.04      0.54      0.07      1978

    accuracy                           0.85    184167
   macro avg       0.52      0.70      0.50    184167
weighted avg       0.98      0.85      0.91    184167



In [0]:
#Accuracy obtained on the training set
cm = confusion_matrix(train_p1, y_test )
acc = cm.diagonal().sum()/cm.sum()
print(acc)

0.8546699462987397


In [0]:
 #Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(train_data, label=train_labels)

#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
     params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              #'min_child_weight' : int(min_child_weight),
              'eta': 0.05,
              'eval_metric': 'auc'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations 
     cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
     return cv_result['test-auc-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0.05,0.3),
                                             'n_estimators':(100,120)
 #'min_child_weight':(1,10)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=2, init_points=3, acq='ei')

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
